### Collecting [CMIP6 Daily Data: Global Gridded Data from 2015-2100](https://www.nccs.nasa.gov/services/data-collections/land-based-products/nex-gddp-cmip6)

base_path = "https://ds.nccs.nasa.gov/thredds/catalog/AMES/NEX/GDDP-CMIP6/ACCESS-CM2/ssp126/r1i1p1f1/pr/catalog.html?dataset=AMES/NEX/GDDP-CMIP6/ACCESS-CM2/ssp126/r1i1p1f1/pr/pr_day_ACCESS-CM2_ssp126_r1i1p1f1_gn_2015_v1.1.nc"

In [2]:
!pip install requests

  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached charset_normalizer-3.4.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (35 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
Using cached requests-2.32.3-py3-none-any.whl (64 kB)
Using cached certifi-2025.1.31-py3-none-any.whl (166 kB)
Using cached charset_normalizer-3.4.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (143 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)


In [1]:
import os
import requests

# Configurations
scenarios = ['ssp126', 'ssp585']
vars = ['tasmin']
years = [2015, 2060, 2100]
base_year = 2015  # Year to determine the correct version

models = [
    "ACCESS-CM2", "UKESM1-0-LL"
]

# Model-specific keys
model_keys = {
    "ACCESS-CM2": "r1i1p1f1", "UKESM1-0-LL": "r1i1p1f2"
}

model_keys2 = {
    "ACCESS-CM2": "gn", "UKESM1-0-LL": "gn"
}

base_url = "https://ds.nccs.nasa.gov/thredds/fileServer/AMES/NEX/GDDP-CMIP6"

# Function to check if URL exists
def url_exists(url):
    response = requests.head(url)
    return response.status_code == 200

# Function to determine correct version
def find_best_version(model, scenario, var):
    base_path = f"{base_url}/{model}/{scenario}/{model_keys[model]}/{var}/{var}_day_{model}_{scenario}_{model_keys[model]}_{model_keys2[model]}"
    version = 1

    print(f"Checking {model} - {scenario} - {var} for available versions...")

    # Check if _v1.1 exists
    test_url = f"{base_path}_{base_year}_v1.1.nc"
    if url_exists(test_url):
        print(f"Found v1.1 for {model}, checking higher versions...")
        while True:
            next_version = f"{base_path}_{base_year}_v1.{version + 1}.nc"
            if url_exists(next_version):
                version += 1
                print(f"Found v1.{version} for {model}")
            else:
                print(f"Final version selected: v1.{version}")
                return f"_v1.{version}"
    else:
        # Check without version
        test_url = f"{base_path}_{base_year}.nc"
        if url_exists(test_url):
            print(f"Using non-versioned file for {model}")
            return ""
    
    print(f"No valid file found for {model} - {scenario} - {var}")
    return None  # No valid file found

# Main processing
master_log = []
for model in models:
    model_log = []
    summary_log = []

    os.makedirs(f'CMIP6/{model}', exist_ok=True)
    model_file_path = os.path.join(f'CMIP6/{model}', f"{model}_paths.txt")
    summary_file_path = os.path.join(f'CMIP6/{model}', f"{model}_summary.txt")

    for scenario in scenarios:
        for var in vars:
            best_version = find_best_version(model, scenario, var)

            if best_version is not None:
                for year in years:
                    file_name = f"{var}_day_{model}_{scenario}_{model_keys[model]}_{model_keys2[model]}_{year}{best_version}.nc"
                    file_path = os.path.join(model, file_name)

                    if os.path.exists(file_path):
                        print(f"Skipping {file_name}, already exists in {model}/")
                        continue

                    url = f"{base_url}/{model}/{scenario}/{model_keys[model]}/{var}/{file_name}"
                    model_log.append(url)

                if model_log:  # Only append if there's at least one valid URL
                    summary_log.append(f"{model}, {scenario}, {var}: Using version {best_version if best_version else 'none'}")
                    master_log.append(f"{model}, {scenario}, {var}: {model_log[0]} (applied to all years)")
                else:
                    summary_log.append(f"{model}, {scenario}, {var}: No valid URLs found")
            else:
                summary_log.append(f"{model}, {scenario}, {var}: No valid file found")

    print(f"Saving logs for {model}...")
    
    # Write individual model logs
    with open(model_file_path, "w") as model_file:
        model_file.write("\n".join(model_log))

    with open(summary_file_path, "w") as summary_file:
        summary_file.write("\n".join(summary_log))

# Write master log
with open("CMIP6/master_paths.txt", "w") as master_file:
    master_file.write("\n".join(master_log))

print("Processing complete. Logs saved.")


Checking ACCESS-CM2 - ssp126 - tasmin for available versions...
Found v1.1 for ACCESS-CM2, checking higher versions...
Found v1.2 for ACCESS-CM2
Final version selected: v1.2
Checking ACCESS-CM2 - ssp585 - tasmin for available versions...
Using non-versioned file for ACCESS-CM2
Saving logs for ACCESS-CM2...
Checking UKESM1-0-LL - ssp126 - tasmin for available versions...
Found v1.1 for UKESM1-0-LL, checking higher versions...
Found v1.2 for UKESM1-0-LL
Final version selected: v1.2
Checking UKESM1-0-LL - ssp585 - tasmin for available versions...
Using non-versioned file for UKESM1-0-LL
Saving logs for UKESM1-0-LL...
Processing complete. Logs saved.


Launch a tmux session and run:

```bash

cd Coding_Workshop/CMIP6/<Model>

wget -i <Model>_paths.txt 

```